In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090 Laptop GPU. Max memory: 15.992 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_connection = sqlite3.connect('../Database/minifigs.db')

# Query to get the relevant data from your table
query = "SELECT fig_id, description, formatted_parts FROM parsed_minifig_data"

# Load the data into a pandas DataFrame
df = pd.read_sql_query(query, db_connection)

# Close the database connection
db_connection.close()

# Display the data
print(df.head())

       fig_id                               description  \
0  fig-000001                        Toy Store Employee   
1  fig-000002                              Customer Kid   
2  fig-000003                     Assassin Droid, White   
3  fig-000004  Man, White Torso, Black Legs, Brown Hair   
4  fig-000005           Captain America with Short Legs   

                                     formatted_parts  
0  "Minifig Headwear": "Hair Mid-Length Swept Sid...  
1  "Minifig Lower Body": "Legs Short <Blue>"\n"Mi...  
2  "Bars, Ladders and Fences": "Bar 1L with Clip ...  
3  "Minifig Lower Body": "Hips and Black Legs <Bl...  
4  "Minifig Lower Body": "Legs Short <Blue>"\n"Mi...  


In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Assuming you're using a tokenizer that has an EOS token

instructions = []
inputs = []
outputs = []
# Function to format data into prompts
def format_prompts(df):


    for idx, row in df.iterrows():
        instruction = "Based on the provided description of the LEGO minifigure, list and detail all its individual parts along with their colors."
        input_ = row['description']
        output = row['formatted_parts']

        instructions.append(instruction)
        inputs.append(input_)
        outputs.append(output)


    return {"instruction": instructions, "input": inputs, "output": outputs}



# Format your data
formatted_data = format_prompts(df)

In [11]:
from datasets import Dataset, DatasetDict

dataset_dict = {
    "instruction": instructions,
    "input": inputs,
    "output": outputs
}

dataset = Dataset.from_dict(dataset_dict)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 13403
})


Train the Model

In [12]:
from datasets import DatasetDict

# Set a seed for reproducibility
seed = 42

# Split into 85% train, 10% validation, and 10% test
train_test_split = dataset.train_test_split(test_size=0.05, seed=seed)  # 10% for test set
train_valid_split = train_test_split['train'].train_test_split(test_size=0.05, seed=seed)  # 5% of train for validation

# Combine the splits into a DatasetDict
dataset_split = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': train_test_split['test']
})

# Check the splits
print(dataset_split)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 12095
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 637
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 671
    })
})


In [13]:
# Print the number of rows before saving
print(f"Number of rows in dataset before saving: {len(dataset_split)}")

# Save the dataset to disk
dataset_split.save_to_disk('./formatted_minifig_dataset')
print(dataset_split)


Number of rows in dataset before saving: 3


Saving the dataset (0/1 shards):   0%|          | 0/12095 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/637 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/671 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 12095
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 637
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 671
    })
})


In [14]:
from datasets import load_from_disk

# Load the dataset back from disk
dataset = load_from_disk('./formatted_minifig_dataset')["train"]

# Print the loaded dataset structure and number of rows
print(dataset)
print(f"Number of rows in loaded dataset: {len(dataset)}")

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 12095
})
Number of rows in loaded dataset: 12095


In [15]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/12095 [00:00<?, ? examples/s]

In [16]:
print(dataset[0])

{'instruction': 'Based on the provided description of the LEGO minifigure, list and detail all its individual parts along with their colors.', 'input': 'Duke DeTain, Stunt Rider Outfit', 'output': '"Minifig Headwear": "Hair Coiled and Short <Black>"\n"Minifig Lower Body": "Hips and Flat Silver Legs <Flat Silver>"\n"Minifig Upper Body": "Torso Bodysuit, Zipper, Dark Blue Panels, Lime Trim Print, Dark Blue Arms, Dark Bluish Gray Hands <Flat Silver>"\n"Minifig Heads": "Minifig Head, Reflective Sunglasses, Smile with Teeth / White Pupils Print <Yellow>"', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nBased on the provided description of the LEGO minifigure, list and detail all its individual parts along with their colors.\n\n### Input:\nDuke DeTain, Stunt Rider Outfit\n\n### Response:\n"Minifig Headwear": "Hair Coiled and Short <Black>"\n"Minifig Low

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/12095 [00:00<?, ? examples/s]

In [18]:
trainer_stats = None

In [19]:
torch.cuda.empty_cache()

In [20]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090 Laptop GPU. Max memory = 15.992 GB.
5.984 GB of memory reserved.


In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 12,095 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 3,024
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.043800
2,2.896000
3,2.911900
4,2.872900
5,2.666800
6,2.591300
7,2.249800
8,1.974800
9,1.809900
10,1.622900


In [24]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

6391.9633 seconds used for training.
106.53 minutes used for training.
Peak reserved memory = 8.145 GB.
Peak reserved memory for training = 2.161 GB.
Peak reserved memory % of max memory = 50.932 %.
Peak reserved memory for training % of max memory = 13.513 %.


In [31]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Based on the provided description of the LEGO minifigure, list and detail all its individual parts along with their colors.", # instruction
        "Joker and batman's child", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nBased on the provided description of the LEGO minifigure, list and detail all its individual parts along with their colors.\n\n### Input:\nJoker and batman\'s child\n\n### Response:\n"Minifig Lower Body": "Legs Short <Blue>"\n"Minifig Upper Body": "Torso, Dual Molded Arms, Batman Logo in Black Oval with Yellow Clasps Print, Blue Sleeves Pattern, Light Nougat Arms, Black Hands <Blue>"\n"Minifig Heads": "Minifig Head Joker, Red Eyebrows, Green Eyeshadow, Bright Light Orange Cheek Lines, Wide Open Mouth Grin, Red Lips <Light Nougat>"\n"Minifig Headwear": "Hair and Hat, Floppy Witch Hat with Black Hair Print <Red>"<|end_of_text|>']

In [27]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")


('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
tokenizer.model_max_length